In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

In [9]:
class Discriminator(nn.Module):
  def __init__(self, img_dim):
    super().__init__()
    self.disc = nn.Sequential(
        nn.Linear(img_dim, 128),
        nn.LeakyReLU(0.1),
        nn.Linear(128, 1),
        nn.Sigmoid(),
    )
  def forward(self, x):
    return self.disc(x)

In [10]:
class Generator(nn.Module):
  def __init__(self, z_dim, img_dim):
    super().__init__()
    self.gen = nn.Sequential(
        nn.Linear(z_dim, 256),
        nn.LeakyReLU(0.1),
        nn.Linear(256, img_dim),
        nn.Tanh(),
    )
  def forward(self, x):
    return self.gen(x)

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [12]:
lr = 3e-4
z_dim = 64
image_dim = 28 * 28 * 1
batch_size = 32
num_epochs = 50

In [13]:
disc = Discriminator(image_dim).to(device)
gen = Generator(z_dim, image_dim).to(device)
fixed_noise = torch.randn((batch_size, z_dim)).to(device)
transforms = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.1207,), (0.3081,))]
)

In [14]:
dataset = datasets.MNIST(root = "dataset/", transform = transforms, download = True)
loader = DataLoader(dataset, batch_size = batch_size, shuffle = True)
opt_disc = optim.Adam(disc.parameters(), lr = lr)
opt_gen = optim.Adam(gen.parameters(), lr = lr)
criterion = nn.BCELoss()
writer_fake = SummaryWriter(f'runs/GAN_MNIST/fake')
writer_real = SummaryWriter(f'runs/GAN_MNIST/real')
step = 0

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:02<00:00, 4766578.78it/s]


Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 125932.33it/s]


Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:06<00:00, 237571.93it/s]


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 4351422.74it/s]

Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



In [15]:
for epoch in range(num_epochs):
  for batch_idx, (real, _) in enumerate(loader):
    real = real.view(-1, 784).to(device)
    batch_size = real.shape[0]

    noise = torch.randn(batch_size, z_dim).to(device)
    fake = gen(noise)
    disc_real = disc(real).view(-1)
    lossD_real = criterion(disc_real, torch.ones_like(disc_real))
    disc_fake = disc(fake).view(-1)
    lossD_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
    lossD = (lossD_real + lossD_fake) / 2
    disc.zero_grad()
    lossD.backward(retain_graph = True)
    opt_disc.step()

    output = disc(fake).view(-1)
    lossG = criterion(output, torch.ones_like(output))
    gen.zero_grad()
    lossG.backward()
    opt_gen.step()

    if batch_idx == 0:
      print(
          f"Epoch [{epoch}/{num_epochs}] Batch {batch_idx}/{len(loader)} \
          Loss D: {lossD:.4f}, loss G: {lossG:.4f}")
      with torch.no_grad():
        fake = gen(fixed_noise).reshape(-1, 1, 28, 28)
        data = real.reshape(-1, 1, 28, 28)
        img_grid_fake = torchvision.utils.make_grid(fake, normalize = True)
        img_grid_real = torchvision.utils.make_grid(data, normalize = True)

        writer_fake.add_image(
            "Mnist Fake Images", img_grid_fake, global_step = step
        )
        writer_real.add_image(
            "Mnist Real Images", img_grid_real, global_step = step
        )
        step += 1

Epoch [0/50] Batch 0/1875           Loss D: 0.7248, loss G: 0.6886
Epoch [1/50] Batch 0/1875           Loss D: 0.2676, loss G: 1.5444
Epoch [2/50] Batch 0/1875           Loss D: 0.0887, loss G: 3.2111
Epoch [3/50] Batch 0/1875           Loss D: 0.0240, loss G: 4.2945
Epoch [4/50] Batch 0/1875           Loss D: 0.1139, loss G: 4.0903
Epoch [5/50] Batch 0/1875           Loss D: 0.0538, loss G: 4.4003
Epoch [6/50] Batch 0/1875           Loss D: 0.0471, loss G: 5.6374
Epoch [7/50] Batch 0/1875           Loss D: 0.0319, loss G: 3.9922
Epoch [8/50] Batch 0/1875           Loss D: 0.0119, loss G: 5.2854
Epoch [9/50] Batch 0/1875           Loss D: 0.0568, loss G: 5.2988
Epoch [10/50] Batch 0/1875           Loss D: 0.0159, loss G: 5.3584
Epoch [11/50] Batch 0/1875           Loss D: 0.0141, loss G: 5.9824
Epoch [12/50] Batch 0/1875           Loss D: 0.0054, loss G: 5.6700
Epoch [13/50] Batch 0/1875           Loss D: 0.0285, loss G: 5.0311
Epoch [14/50] Batch 0/1875           Loss D: 0.0922, loss 